## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Import the NumPy module.
import numpy as np

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bara,NG,10.3744,10.7288,105.03,16,43,0.98,scattered clouds
1,1,The Valley,AI,18.2170,-63.0578,82.54,61,40,19.57,scattered clouds
2,2,Cidreira,BR,-30.1811,-50.2056,71.82,83,97,9.15,overcast clouds
3,3,Dikson,RU,73.5069,80.5464,9.59,95,100,4.59,overcast clouds
4,4,Saint-Pierre,RE,-21.3393,55.4781,80.29,69,100,19.57,overcast clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 78


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
filtered_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Cidreira,BR,-30.1811,-50.2056,71.82,83,97,9.15,overcast clouds
6,6,Castro,BR,-24.7911,-50.0119,75.07,65,95,5.37,overcast clouds
7,7,Port Elizabeth,ZA,-33.9180,25.5701,67.10,82,79,11.99,broken clouds
9,9,Airai,TL,-8.9266,125.4092,59.22,83,97,2.62,overcast clouds
11,11,Busselton,AU,-33.6500,115.3333,59.49,56,99,5.26,overcast clouds
12,12,Ciulnita,RO,44.5333,27.3500,75.60,33,4,15.48,clear sky
19,19,Henties Bay,NaN,-22.1160,14.2845,65.57,69,1,10.65,clear sky
25,25,Marsa Matruh,EG,31.3525,27.2453,65.21,57,4,10.71,clear sky
26,26,East London,ZA,-33.0153,27.9116,66.22,83,100,15.17,light rain
27,27,Ancud,CL,-41.8697,-73.8203,52.23,94,100,7.16,moderate rain


In [13]:
# 4a. Determine if there are any empty rows.
filtered_df.isnull().sum()


City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cidreira,BR,71.82,overcast clouds,-30.1811,-50.2056,
6,Castro,BR,75.07,overcast clouds,-24.7911,-50.0119,
7,Port Elizabeth,ZA,67.10,broken clouds,-33.9180,25.5701,
9,Airai,TL,59.22,overcast clouds,-8.9266,125.4092,
11,Busselton,AU,59.49,overcast clouds,-33.6500,115.3333,
12,Ciulnita,RO,75.60,clear sky,44.5333,27.3500,
25,Marsa Matruh,EG,65.21,clear sky,31.3525,27.2453,
26,East London,ZA,66.22,light rain,-33.0153,27.9116,
27,Ancud,CL,52.23,moderate rain,-41.8697,-73.8203,
34,Bethel,US,53.02,few clouds,41.3712,-73.4140,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] =f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cidreira,BR,71.82,overcast clouds,-30.1811,-50.2056,Hotel Castelo
6,Castro,BR,75.07,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
7,Port Elizabeth,ZA,67.10,broken clouds,-33.9180,25.5701,39 On Nile Guest House
11,Busselton,AU,59.49,overcast clouds,-33.6500,115.3333,Observatory Guest House
12,Ciulnita,RO,75.60,clear sky,44.5333,27.3500,Hotel Select
25,Marsa Matruh,EG,65.21,clear sky,31.3525,27.2453,Beau Site
26,East London,ZA,66.22,light rain,-33.0153,27.9116,Tu Casa
27,Ancud,CL,52.23,moderate rain,-41.8697,-73.8203,Hotel Arena Gruesa
34,Bethel,US,53.02,few clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
39,Ribeira Grande,PT,54.00,broken clouds,38.5167,-28.7000,Quinta da Meia Eira


In [21]:
# 8a. Create the output File (CSV)WeatherPy_vacation.csv
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(marker_layer)
fig.add_layer(heat_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))